# Catboost
catboost is a machine learning library for classification and regression problems. It provides a set of tools for training, evaluating, and using classification and regression models. Catboost is a popular machine learning library for binary and multi-class classification tasks.  

In [14]:
# install catboost
!pip install catboost -q


In [15]:
# install libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier


In [16]:
# load the dataset
df= sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Pre-Processing the dataset

In [17]:
# impute missing values by using KNN imputer in fare, age , 
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df['age']= imputer.fit_transform(df[['age']])

# impute missing value of embarked and embark_town using pandas
df['embarked']= df['embarked'].fillna(df['embarked'].mode()[0])
df['embark_town']= df['embark_town'].fillna(df['embark_town'].mode()[0])

# droping the deck coloumn
df.drop('deck',axis=1, inplace=True)


In [18]:
#check for null values
df.isnull().sum().sort_values(ascending=False)

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     891 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  embark_town  891 non-null    object  
 12  alive        891 non-null    object  
 13  alone        891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(5)
memory usage: 79.4+ KB


In [28]:
# # convert the each columnn tye to category using for loop
# for col in df.columns:
#     if df[col].dtypes=='object' or df[col].dtypes=='category':
#         df[col]=df[col].astype('category')

catogorial_cols=df.select_dtypes(include=['object','category']).columns
df[catogorial_cols]=df[catogorial_cols].astype('category')

In [21]:
# split data into X and y
X=df.drop('survived',axis=1)
y=df['survived']

# split data into train and test
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3,random_state=42)


In [30]:
# run the catboost classifier
model=CatBoostClassifier(iterations=100, depth=3, learning_rate=0.1, loss_function='Logloss', 
                         eval_metric='Accuracy', random_seed=42, verbose=False)

# fit the model 
model.fit(X_train,y_train, catogorial_cols.to_list())

# make predictions
y_pred=model.predict(X_test)

# evaluate the model
accuracy=accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)
print('F1 Score:', f1_score(y_test, y_pred))
print('Classification Report:', classification_report(y_test, y_pred))


Accuracy: 1.0
F1 Score: 1.0
Classification Report:               precision    recall  f1-score   support

           0       1.00      1.00      1.00       157
           1       1.00      1.00      1.00       111

    accuracy                           1.00       268
   macro avg       1.00      1.00      1.00       268
weighted avg       1.00      1.00      1.00       268

